In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<h2>Data - Loading</h2>

In [56]:
#load data
df = pd.read_pickle('df_merged2.pickle')

#top brands data only
df_topbrands = df['Product'].str.contains('Gardein', case=False) | df['Product'].str.contains('Morningstar', case=False) | df['Product'].str.contains('Beyond Meat', case=False) | df['Product'].str.contains('Impossible', case=False) | df['Product'].str.contains('Private Label', case=False)
df_topbrands = df[df_topbrands]

In [57]:
#drop multiple columns
cols = df_topbrands.columns
special_cols = cols[cols.str.contains('Special')]

#remove special columns and Meat Source
df_topbrands = df_topbrands.drop(columns=special_cols)
df_topbrands = df_topbrands.drop(columns=['Meat Source','UPC 13 digit_x','UPC 13 digit_y'])

<h2>Cleaning & Imputation</h2>

In [58]:
def fill_missing_with_ratio(df, col1, col2):
    # Calculate the ratio for non-missing values
    valid_ratios = df.dropna(subset=[col2])[col2] / df.dropna(subset=[col2])[col1]
    mean_ratio = valid_ratios.mean()
    # Fill missing values in col2 using the mean ratio multiplied by col1
    df[col2] = df.apply(lambda row: mean_ratio * row[col1] if pd.isna(row[col2]) else row[col2], axis=1)

    return df

In [59]:
def substitute_missing_values(df, source_col, target_col):
    # Fill missing values in target_col with values from source_col
    df[target_col] = df[target_col].fillna(df[source_col])
    return df

In [60]:
#cleaning and imputation
df_nomiss = fill_missing_with_ratio(df,"Unit Sales", "Unit Sales Any Merch")
df_nomiss = fill_missing_with_ratio(df_nomiss,"Volume Sales", "Volume Sales Any Merch")
df_nomiss = fill_missing_with_ratio(df_nomiss,"Dollar Sales", "Dollar Sales Any Merch")
df_nomiss = fill_missing_with_ratio(df_nomiss,"Price per Unit", "Price per Unit Any Merch")
df_nomiss = fill_missing_with_ratio(df_nomiss,"Price per Volume", "Price per Volume Any Merch")
df_nomiss = fill_missing_with_ratio(df_nomiss,"ACV Weighted Distribution", "ACV Weighted Distribution Any Merch")
df_nomiss = substitute_missing_values(df_nomiss, "Unit Sales", "Unit Sales No Merch")
df_nomiss = substitute_missing_values(df_nomiss, "Volume Sales", "Volume Sales No Merch")
df_nomiss = substitute_missing_values(df_nomiss, "Dollar Sales", "Dollar Sales No Merch")
df_nomiss = substitute_missing_values(df_nomiss, "Price per Unit", "Price per Unit No Merch")
df_nomiss = substitute_missing_values(df_nomiss, "Price per Volume", "Price per Volume No Merch")
df_nomiss = substitute_missing_values(df_nomiss, "ACV Weighted Distribution", "ACV Weighted Distribution No Merch")

In [61]:
special_cols = cols[cols.str.contains('Reductions') | cols.str.contains('Feature Only') | cols.str.contains('Display Only') | cols.str.contains('Feature and Display')| cols.str.contains('Incremental') | cols.str.contains('Type Of')]
df_nomiss = df_nomiss.drop(columns=special_cols)

<h2>Exploratory Data Analysis</h2>
<h4>Analysis by Top Products</h4>

In [62]:
df_bacon = df_nomiss[df_nomiss['Product'].str.contains('Bacon', case=False)]
df_bacon_nodup = df_bacon.drop_duplicates(subset='Product')
#top 10 products from df_bacon for each brand
top_bacon_products_by_brand = df_bacon_nodup.groupby('Brand Name').apply(
    lambda x: x.nlargest(10, 'Dollar Sales')).reset_index(drop=True)


C:\Users\varun\AppData\Local\Temp\ipykernel_26512\3755464178.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_bacon_products_by_brand = df_bacon_nodup.groupby('Brand Name').apply(


In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
aggregated_data = (
    top_bacon_products_by_brand.groupby(['Brand Name', 'Product'])
    .agg({'Price per Unit': 'mean'})
    .sort_values(by=['Brand Name', 'Price per Unit'], ascending=[True, False])
)

# Reset the index to make 'Brand Name' and 'Product' regular columns
aggregated_data.reset_index(inplace=True)

# Filter top 10 products for each brand
top_products = aggregated_data.groupby('Brand Name').head(10)

# Plotting
plt.figure(figsize=(20, 10))
barplot = sns.barplot(
    data=top_products,
    x='Brand Name',
    y='Price per Unit',
    hue='Product',
    dodge=True  # This places all bars side by side
)

# Adding labels and title
plt.xlabel('Brand Names')
plt.ylabel('Price per Unit')
plt.title('Top 10 Bacon Products vs Price per Unit by Brand')
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout to make room for legend
plt.show()

In [ ]:
import seaborn as sns
aggregated_data = (
    top_bacon_products_by_brand.groupby(['Brand Name', 'Product'])
    .agg({'Price per Ounce': 'mean'})
    .sort_values(by=['Brand Name', 'Price per Ounce'], ascending=[True, False])
)

# Reset the index to make 'Brand Name' and 'Product' regular columns
aggregated_data.reset_index(inplace=True)

# Filter top 10 products for each brand
top_products = aggregated_data.groupby('Brand Name').head(10)

# Plotting
plt.figure(figsize=(20, 10))
barplot = sns.barplot(
    data=top_products,
    x='Brand Name',
    y='Price per Ounce',
    hue='Product',
    dodge=True 
)

# Adding labels and title
plt.xlabel('Brand Names')
plt.ylabel('Price per Ounce')
plt.title('Top 10 Bacon Products vs Price per Ounce by Brand')
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout to make room for legend
plt.show()

In [ ]:
import seaborn as sns
aggregated_data = (
    top_bacon_products_by_brand.groupby(['Brand Name', 'Product'])
    .agg({'Price per Volume': 'mean'})
    .sort_values(by=['Brand Name', 'Price per Volume'], ascending=[True, False])
)

# Reset the index to make 'Brand Name' and 'Product' regular columns
aggregated_data.reset_index(inplace=True)

# Filter top 10 products for each brand
top_products = aggregated_data.groupby('Brand Name').head(10)

# Plotting
plt.figure(figsize=(20, 10))
barplot = sns.barplot(
    data=top_products,
    x='Brand Name',
    y='Price per Volume',
    hue='Product',
    dodge=True  
)


plt.xlabel('Brand Names')
plt.ylabel('Price per Volume')
plt.title('Top 10 Bacon Products vs Price per Volume by Brand')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

<h4>Analysis based on Region</h4>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


melted_df = df_bacon.melt(id_vars=['Geography', 'Brand Name'], 
                    value_vars=['Dollar Sales Any Merch', 'Dollar Sales No Merch'],
                    var_name='Sales Type', value_name='Dollar_Sales')

# Create a new column 'Geo-Brand' to combine 'Geography' and 'Brand Name' for x-axis labeling
melted_df['Geo-Brand'] = melted_df['Geography'] + " - " + melted_df['Brand Name']

# Get unique geographies to create separate plots
geographies = melted_df['Geography'].unique()

# Loop through each geography to create a separate plot
for geo in geographies:
    plt.figure(figsize=(18, 10))
    # Filter the data for the current geography
    geo_data = melted_df[melted_df['Geography'] == geo]
    sns.barplot(
        data=geo_data,
        x='Geo-Brand',
        y='Dollar_Sales',
        hue='Sales Type',  
        palette='viridis'
    )
    plt.xlabel('Geography - Brand')
    plt.ylabel('Dollar_Sales')
    plt.title(f'Comparison of Dollar Sales With and Without Merchandising in {geo}')
    plt.xticks(rotation=45)
    plt.legend(title='Sales Type')
    plt.tight_layout()

    plt.show()


<h2>OLS Regressions</h2>
<h4>OLS Regressions based on Bacon Products</h4>

In [ ]:
import statsmodels.formula.api as smf

model_data = df_bacon.copy()

#round price per unit to 2 decimal
model_data['Price per Volume No Merch'] = model_data['Price per Volume No Merch'].round(2)
model_data['Volume Sales No Merch'] = model_data['Volume Sales No Merch'].round(2)
model_data['Dollar Sales No Merch'] = model_data['Dollar Sales No Merch'].round(2)
model_data['Flavor / Scent'] = model_data['Flavor / Scent']

model_data.rename(columns={
    'Dollar Sales No Merch':'Dollar_Sales_No_Merch',
    'Price per Volume No Merch':'Price_per_Volume_No_Merch',
    'Price per Unit Any Merch':'Price_per_Unit_Any_Merch',
    'Price per Unit': 'Price_per_Unit',
    'Flavor / Scent':'Flavor_Scent',
    'Volume Sales No Merch':'Volume_Sales_No_Merch',
    'Price per Ounce':'Price_per_Ounce',
    'Dollar Sales Any Merch' :'Dollar_Sales_Any_Merch',
    'Volume Sales' :'Volume_Sales',
    'Price per Volume' :'Price_per_Volume',
    'Unit Sales':'Unit_Sales',
    'Price per Ounce': 'Price_per_Ounce',
    'Brand Name':'Brand_Name'
}, inplace=True)

formula = 'Unit_Sales ~ Price_per_Unit + C(Geography) + Price_per_Unit:C(Geography) \
    + C(Brand_Name):C(Geography) + C(Brand_Name):Price_per_Unit '
model = smf.ols(formula=formula, data=model_data).fit()
model.summary()

<h4>OLS Based on Flavors</h4>

In [ ]:
flavor = ['XXXX']
#Different flavours were pulled from the data and a string containing all the flavours (comma seperated) was assinged to this variable

In [ ]:
df_flavor = df_nomiss[df_nomiss['Flavor / Scent'].isin(flavor)]
df_price_flavor = df_flavor[(df_flavor['Price per Unit'] >= 13) & (df_flavor['Price per Unit'] <= 15.1)]
model_data = df_price_flavor.copy()

#round price per unit to 2 decimal
model_data['Price per Volume No Merch'] = model_data['Price per Volume No Merch'].round(2)
model_data['Volume Sales No Merch'] = model_data['Volume Sales No Merch'].round(2)
model_data['Dollar Sales No Merch'] = model_data['Dollar Sales No Merch'].round(2)
model_data['Flavor / Scent'] = model_data['Flavor / Scent']

model_data.rename(columns={
    'Dollar Sales No Merch':'Dollar_Sales_No_Merch',
    'Price per Volume No Merch':'Price_per_Volume_No_Merch',
    'Price per Unit Any Merch':'Price_per_Unit_Any_Merch',
    'Price per Unit': 'Price_per_Unit',
    'Flavor / Scent':'Flavor_Scent',
    'Volume Sales No Merch':'Volume_Sales_No_Merch',
    'Price per Ounce':'Price_per_Ounce',
    'Dollar Sales Any Merch' :'Dollar_Sales_Any_Merch',
    'Volume Sales' :'Volume_Sales',
    'Price per Volume' :'Price_per_Volume',
    'Unit Sales':'Unit_Sales',
    'Price per Ounce': 'Price_per_Ounce',
    'Dollar Sales':'Dollar_Sales'
}, inplace=True)

formula = 'Dollar_Sales ~ Price_per_Ounce + Price_per_Ounce:C(Geography)+\
      C(Flavor_Scent) + Price_per_Ounce:C(Flavor_Scent) + \
        Price_per_Ounce:C(Geography):C(Flavor_Scent)+I(Price_per_Unit**2)'


model = smf.ols(formula=formula, data=model_data).fit()
model.summary()

<h4>OLS Based on Form</h4>

In [ ]:
df2 = df_nomiss[(df_nomiss['Price per Unit'] >= 7) & (df_nomiss['Price per Unit'] <= 9) & df_nomiss['Product'].str.contains('Gardein', case=False)]

model_data = df2.copy()

#round price per unit to 2 decimal
model_data['Price per Volume No Merch'] = model_data['Price per Volume No Merch'].round(2)
model_data['Volume Sales No Merch'] = model_data['Volume Sales No Merch'].round(2)
model_data['Dollar Sales No Merch'] = model_data['Dollar Sales No Merch'].round(2)
model_data['Flavor / Scent'] = model_data['Flavor / Scent']

model_data.rename(columns={
    'Dollar Sales No Merch':'Dollar_Sales_No_Merch',
    'Price per Volume No Merch':'Price_per_Volume_No_Merch',
    'Price per Unit Any Merch':'Price_per_Unit_Any_Merch',
    'Price per Unit': 'Price_per_Unit',
    'Flavor / Scent':'Flavor_Scent',
    'Volume Sales No Merch':'Volume_Sales_No_Merch',
    'Price per Ounce':'Price_per_Ounce',
    'Dollar Sales Any Merch' :'Dollar_Sales_Any_Merch',
    'Volume Sales' :'Volume_Sales',
    'Price per Volume' :'Price_per_Volume',
    'Unit Sales':'Unit_Sales',
    'Price per Ounce': 'Price_per_Ounce',
    'Dollar Sales':'Dollar_Sales',
    'Brand Name':'Brand_Name'
}, inplace=True)

# formula = 'Dollar_Sales ~ Price_per_Ounce + Price_per_Ounce:C(Form) + \
#     Price_per_Ounce:C(Geography):C(Form)+I(Price_per_Unit**2)  + C(Form):C(Brand_Name):C(Geography)'
formula = 'Dollar_Sales ~ Price_per_Unit + Price_per_Unit:C(Form) + \
    Price_per_Unit:C(Geography):C(Form)  + C(Form):C(Geography)'


model = smf.ols(formula=formula, data=model_data).fit()
model.summary()